<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/rags/evaluate-rags-rigorously-or-perish.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# Installing Python packages & hiding
!pip install --quiet \
  chromadb \
  datasets \
  langchain \
  polars \
  ragas
  # arxiv \
  # pytube \
  # pymupdf \
  # xmltodict \
  # youtube-transcript-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 745.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
# Importing the packages

import os

import chromadb
from datasets import load_dataset

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import polars as pl
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context


In [16]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as environmental variables

COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
  from google.colab import userdata, data_table
  # Secrets
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
  runtime_info = "Colab runtime"

  # Enabling Colab's data formatter for pandas
  data_table.enable_dataframe_formatter()
else:
  # Secrets
  OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
  runtime_info = "Non Colab runtime"

print(runtime_info)

Colab runtime


In [57]:
# Loading small sample of article from CNN and Daily Mail news dateset on HF: https://huggingface.co/datasets/cnn_dailymail
# - Not directly via LangChain with HuggingFaceDatasetLoader class because, it doesn't have split argument

interim = load_dataset(path="cnn_dailymail", name='1.0.0', split='train[:100]')
news_pl = pl.from_arrow(interim.data.table)
news_pd = news_pl.to_pandas()
from langchain_community.document_loaders import PolarsDataFrameLoader
loader = PolarsDataFrameLoader(news_pl, page_content_column="article")
news = loader.load()


In [59]:
if COLAB:
  display(news_pd)

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
...,...,...,...
95,"DENVER, Colorado -- A Colorado man terrorized ...",Some witnesses say Colorado does nothing to pr...,f70a7abb6c5b0ef383ea12a4d9ca046a5bd854e5
96,"LONDON, England (CNN) -- Previously unseen foo...",NEW: Jury shown new footage of Diana taken hou...,a3dd38ec7bc9d7e8423b96d8fd0641a2a5d5c984
97,WASHINGTON (CNN) -- Republicans reacted with s...,"Republican Sen. Lindsey Graham: ""I am astounde...",654c6b29b96d2a5a818d91400c20f838b0e8b6df
98,"ST. PETERSBURG, Florida (CNN) -- The acrimony ...","YouTube questions address taxes, the Bible, ab...",764d9ce99a1e3f79d95fbc4b68adbce14e7f8bcd


# Youtube transcripts data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00


,publish_date,author,title,video_url,view_count,length_in_seconds,length_in_words,page_content
0,2024-04-16 00:00:00,BBC News,Tesla whistleblower says she wants an Elon Mus...,https://youtu.be/YEHA-u8b43A,68911,218,531,well the Tesla now and a whistle blow whistleb...
1,2024-04-16 00:00:00,BBC News,UK smoking ban: MPs to vote on banning young p...,https://youtu.be/TlFfHjOMSXQ,34575,192,513,smoking is the single biggest cause of prevent...
2,2024-04-16 00:00:00,BBC News,Right-wing event in Brussels told to shut down...,https://youtu.be/-kZolk9EoMA,30579,181,592,the organizers of a conference in Brussels whi...
3,2024-04-16 00:00:00,BBC News,Israel demands sanctions on Iranian missile pr...,https://youtu.be/JQMZkrz6X08,162122,458,1074,and we begin with events in the Middle East wh...
4,2024-04-16 00:00:00,BBC News,Olympic flame lit in Greece's ancient Olympia ...,https://youtu.be/D1iMZaLjBU4,59247,695,1062,now I want to take you back to Greece and to O...
5,2024-04-11 00:00:00,BBC News,Elon Musk’s AI chatbot generated disinformatio...,https://youtu.be/Sqcv9lCADxE,56472,895,2875,you're watching the context it's time for our ...
6,2024-04-04 00:00:00,BBC News,Taiwan earthquake: More than 600 stranded a da...,https://youtu.be/Sshl0SFO4ZI,159726,445,1243,let's turn to Taiwan rescue efforts are underw...
7,2024-04-03 00:00:00,BBC News,Uganda’s top appeals court rejects petition to...,https://youtu.be/mpOyMOZWEcU,31387,219,596,to Uganda next and the country's constitutiona...
8,2024-04-11 00:00:00,BBC News,US President Joe Biden vows 'ironclad' support...,https://youtu.be/Cg3YMWcjLI4,111802,353,1005,hello I'm Nikki schill welcome to the program ...
9,2024-04-09 00:00:00,BBC News,Lord Cameron meets Antony Blinken after 'priva...,https://youtu.be/yQIMSv9Luw4,44581,484,1440,right we're going to look now at UK US diploma...


In [213]:
# Example LangChain Document object
bbc_news[19]

Document(page_content="3 months after a door plug blew off a Boeing plane mid-flight the US Airline maker is facing increased scrutiny well now the Federal Aviation Administration is investigating a whistleblower's claims that the company took shortcuts that impacted quality and safety our North America business correspondent Aon Delmore has the latest a former Boeing Employee who worked on the 787 accused the company of cutting Corners during production and overlooking engineering problems that could weak in the plane's structural Integrity Boeing dismissed the claims as inaccurate and reiterated confidence in the 787 Dreamliner meanwhile the company delivered its lowest number of planes in the first quarter of the year since 2021 just 83 planes down from 157 the quarter prior Boeing Executives say the company is slowing down production so that it can improve quality control but delivery delays are sparking criticism from customers who have little choice to fulfill their needs Boeing 

In [ ]:
client = chromadb.Client()


In [202]:
base_docs = bbc_news[19:20]

# generator with openai models
llm = ChatOpenAI(model="gpt-3.5-turbo")
generator_llm = llm
critic_llm = llm
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 1,
}




# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(base_docs, 1, distributions)
testset.to_pandas()

embedding nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating:   0%|          | 0/1 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the recent challenges faced by Boeing...,[3 months after a door plug blew off a Boeing ...,The recent challenges faced by Boeing in relat...,simple,"[{'source': 'Iz-XY6XfXjk', 'title': 'Boeing hi...",True


In [221]:
ts_ds = testset.to_dataset()
ts_ds = ts_ds.add_column("answer", ts_ds["ground_truth"])

In [224]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness
)

result = evaluate(
    ts_ds,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness
    ],
)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

In [225]:
result

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9852, 'context_recall': 1.0000, 'answer_correctness': 1.0000}

In [236]:
from langchain.vectorstores import Chroma


client = chromadb.EphemeralClient()
full_text_collection = client.get_or_create_collection(name="full_text")
full_text_collection.add(
    documents=,
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]
)









In [183]:
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter


# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [186]:
embeddings = OpenAIEmbeddings()
new_client = chromadb.EphemeralClient()
openai_lc_client = Chroma.from_documents(
    bbc_news, embeddings, client=new_client, collection_name="no_split"
)



query = "What did the president say about Ketanji Brown Jackson"
docs = openai_lc_client.similarity_search(query)
print(docs[0].page_content)

hello I'm Nikki schill welcome to the program we start this hour with the latest developments in the Middle East President Biden has promised Israel Ironclad United States support amid fears that Iran could launch reprisals for an attack that killed senior Iranians Us Media are reporting that an American General visit Israel on Thursday to discuss Washington's fears President Biden warned at Iran is threatening to launch a significant attack after Israel struck the Iranian Consulate in Syria 10 days ago Mr Biden was speaking hours after the Iranian supreme leader again said Israel would be punished for the strike on its consulate meanwhile the Hamas political leader ishmail hania says there'll be no change in the group's demands for a permanent ceas fight in Gaza despite the killing of three of his sons in an Israeli air strike Israeli media is reporting that the Prime Minister Benjamin Netanyahu had not been briefed in advance of the attack this video is believed to show Ismael hania 

In [233]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(bbc_news)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")